In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/satojkovic/DeepLogo2.git

In [ ]:
import os
os.chdir('DeepLogo2')

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
from tqdm import tqdm

In [ ]:
import gc
gc.collect()

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 8GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=8192)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [ ]:
# def load_image_into_numpy_array(path):
#   img_data = tf.io.gfile.GFile(path, 'rb').read()
#   image = Image.open(BytesIO(img_data))
#   im_width, im_height = image.size
#   return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
def convert_csv_into_numpy_array(csv, im_width, im_height):
  # xmin,ymin,xmax,ymax
  xmin, ymin, xmax, ymax = list(map(int, csv))
  xmin /= im_width
  ymin /= im_height
  xmax /= im_width
  ymax /= im_height
  return np.array([[ymin, xmin, ymax, xmax]], dtype=np.float32)

In [ ]:
def plot_detections(image_np, boxes, classes, scores, category_index, figsize=(12, 16), image_name=None):
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes, classes, scores, category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8
  )
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
%%bash
# wget http://image.ntua.gr/iva/datasets/flickr_logos/flickr_logos_27_dataset.tar.gz
cp /content/drive/MyDrive/DeepLogo2/train_data/flickr_logos_27_dataset.tar.gz .
tar zxvf flickr_logos_27_dataset.tar.gz
cd flickr_logos_27_dataset
tar zxvf flickr_logos_27_dataset_images.tar.gz
cd ..

In [ ]:
%%bash
python preproc_annot.py

## Load and visualize train images

In [ ]:
train_annot_csv = 'flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation_cropped.txt'
train_img_dir = 'flickr_logos_27_dataset/flickr_logos_27_dataset_images'
train_img_npy = '/content/drive/MyDrive/DeepLogo2/train_data/train_images_np.npy'
gt_boxes_npy = '/content/drive/MyDrive/DeepLogo2/train_data/gt_boxes.npy'
gt_class_ids_npy = '/content/drive/MyDrive/DeepLogo2/train_data/gt_class_ids.npy'

train_images_np = []# if not os.path.exists(train_img_npy) else np.load(train_img_npy, allow_pickle=True)
gt_boxes = []# if not os.path.exists(gt_boxes_npy) else np.load(gt_boxes_npy, allow_pickle=True)
gt_class_ids = []# if not os.path.exists(gt_class_ids_npy) else np.load(gt_class_ids_npy, allow_pickle=True)

if len(train_images_np) == 0 or len(gt_boxes) == 0 or len(gt_class_ids) == 0:
  csvs = np.loadtxt(train_annot_csv, dtype=str, delimiter=',')
  for csv in tqdm(csvs):
    if csv[-1] != '0':
      continue
    img_fname = csv[0]
    with tf.io.gfile.GFile(os.path.join(train_img_dir, img_fname), 'rb') as fid:
      encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    train_images_np.append(load_image_into_numpy_array(os.path.join(train_img_dir, img_fname)))
    gt_boxes.append(convert_csv_into_numpy_array(csv[1:-1], width, height))
    class_id = int(csv[-1])
    gt_class_ids.append(class_id)

NUM_TRAIN_SAMPLES = 5#len(train_images_np)
train_images_np = train_images_np[:NUM_TRAIN_SAMPLES]
gt_boxes = gt_boxes[:NUM_TRAIN_SAMPLES]
gt_class_ids = gt_class_ids[:NUM_TRAIN_SAMPLES]

plt.rcParams['axes.grid'] = False
plt.rcParams['figure.figsize'] = [12, 5]

for idx, train_image_np in enumerate(train_images_np):
  plt.subplot(1, 5, idx + 1)
  plt.imshow(train_image_np)
plt.show()

### Assign class category index

In [ ]:
adidas_class_id = 1

category_index = {
    adidas_class_id : {'id' : adidas_class_id, 'name' : 'adidas'}}

# Specify the number of classes that the model will predict
num_classes = 1

### Visualize ground truth bbox

In [ ]:
# give boxes a score of 100%
dummy_scores = np.array([1.0], dtype=np.float32)

# define the figure size
plt.figure(figsize=(30, 15))

# use the `plot_detections()` utility function to draw the ground truth boxes
for idx in range(5):
    plt.subplot(1, 5, idx+1)
    plot_detections(
      train_images_np[idx],
      gt_boxes[idx],
      np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      dummy_scores, category_index)

plt.show()

In [ ]:
# class_names = [
#   "Adidas", 
#   # "Apple", "BMW", "Citroen", "Cocacola",
#   # "DHL", "Fedex", "Ferrari", "Ford", "Google", 
#   # "HP", "Heineken", "Intel", "McDonalds", "Mini", 
#   # "Nbc", "Nike", "Pepsi", "Porsche", "Puma", 
#   # "RedBull", "Sprite", "Starbucks", "Texaco", "Unicef",
#   # "Vodafone", "Yahoo"
# ]

In [ ]:
# def create_category_index(class_names):
#   category_index = {}
#   for i, class_name in enumerate(class_names):
#     category_index[i+1] = {'id': i + 1, 'name': class_name}
#   return category_index

In [ ]:
# category_index = create_category_index(class_names)
# num_classes = len(category_index)

In [ ]:
# import pprint
# pprint.pprint(num_classes)
# pprint.pprint(category_index)

In [ ]:
# gt_boxes = [
#             np.array([[0.436, 0.591, 0.629, 0.712]], dtype=np.float32),
#             np.array([[0.539, 0.583, 0.73, 0.71]], dtype=np.float32),
#             np.array([[0.464, 0.414, 0.626, 0.548]], dtype=np.float32),
#             np.array([[0.313, 0.308, 0.648, 0.526]], dtype=np.float32),
#             np.array([[0.256, 0.444, 0.484, 0.629]], dtype=np.float32)
# ]

In [ ]:
# # By convention, our non-background classes start counting at 1.  Given
# # that we will be predicting just one class, we will therefore assign it a
# # `class id` of 1.
# adidas_class_id = 1
# num_classes = 1

# category_index = {adidas_class_id: {'id': adidas_class_id, 'name': 'adidas'}}

# # Convert class labels to one-hot; convert everything to tensors.
# # The `label_id_offset` here shifts all classes by a certain number of indices;
# # we do this here so that the model receives one-hot labels where non-background
# # classes start counting at the zeroth index.  This is ordinarily just handled
# # automatically in our training binaries, but we need to reproduce it here.
# label_id_offset = 1
# train_image_tensors = []
# gt_classes_one_hot_tensors = []
# gt_box_tensors = []
# for (train_image_np, gt_box_np) in zip(
#     train_images_np, gt_boxes):
#   train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
#       train_image_np, dtype=tf.float32), axis=0))
#   gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
#   zero_indexed_groundtruth_classes = tf.convert_to_tensor(
#       np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)
#   gt_classes_one_hot_tensors.append(tf.one_hot(
#       zero_indexed_groundtruth_classes, num_classes))
# print('Done prepping data.')


# By convention, our non-background classes start counting at 1.  Given
# that we will be predicting just one class, we will therefore assign it a
# `class id` of 1.
duck_class_id = 1
num_classes = 1

category_index = {duck_class_id: {'id': duck_class_id, 'name': 'rubber_ducky'}}

# Convert class labels to one-hot; convert everything to tensors.
# The `label_id_offset` here shifts all classes by a certain number of indices;
# we do this here so that the model receives one-hot labels where non-background
# classes start counting at the zeroth index.  This is ordinarily just handled
# automatically in our training binaries, but we need to reproduce it here.
label_id_offset = 1
train_image_tensors = []
gt_classes_one_hot_tensors = []
gt_box_tensors = []
for (train_image_np, gt_box_np) in zip(
    train_images_np, gt_boxes):
  train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
      train_image_np, dtype=tf.float32), axis=0))
  gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
  zero_indexed_groundtruth_classes = tf.convert_to_tensor(
      np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)
  gt_classes_one_hot_tensors.append(tf.one_hot(
      zero_indexed_groundtruth_classes, num_classes))
print('Done prepping data.')


In [ ]:
# dummy_scores = np.array([1.0], dtype=np.float32)  # give boxes a score of 100%

# plt.figure(figsize=(30, 15))
# for idx in range(5):
#   plt.subplot(2, 3, idx+1)
#   plot_detections(
#       train_images_np[idx],
#       gt_boxes[idx],
#       np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
#       dummy_scores, category_index)
# plt.show()

dummy_scores = np.array([1.0], dtype=np.float32)  # give boxes a score of 100%

plt.figure(figsize=(30, 15))
for idx in range(5):
  plt.subplot(2, 3, idx+1)
  plot_detections(
      train_images_np[idx],
      gt_boxes[idx],
      np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      dummy_scores, category_index)
plt.show()

In [ ]:
# # # Download the checkpoint and put it into models/research/object_detection/test_data/

# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
# !tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
# #!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
# #!tar -xf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

# Download the checkpoint and put it into models/research/object_detection/test_data/

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

In [ ]:
# tf.keras.backend.clear_session()

# print('Building model and restoring weights for fine-tuning...', flush=True)
# num_classes = 1
# pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
# checkpoint_path = '/content/DeepLogo2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'

# # Load pipeline config and build a detection model.
# #
# # Since we are working off of a COCO architecture which predicts 90
# # class slots by default, we override the `num_classes` field here to be just
# # one (for our new rubber ducky class).
# configs = config_util.get_configs_from_pipeline_file(pipeline_config)
# model_config = configs['model']
# model_config.ssd.num_classes = num_classes
# model_config.ssd.freeze_batchnorm = True
# detection_model = model_builder.build(
#       model_config=model_config, is_training=True)

# # Set up object-based checkpoint restore --- RetinaNet has two prediction
# # `heads` --- one for classification, the other for box regression.  We will
# # restore the box regression head but initialize the classification head
# # from scratch (we show the omission below by commenting out the line that
# # we would add if we wanted to restore both heads)
# fake_box_predictor = tf.compat.v2.train.Checkpoint(
#     _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
#     # _prediction_heads=detection_model._box_predictor._prediction_heads,
#     #    (i.e., the classification head that we *will not* restore)
#     _box_prediction_head=detection_model._box_predictor._box_prediction_head,
#     )
# fake_model = tf.compat.v2.train.Checkpoint(
#           _feature_extractor=detection_model._feature_extractor,
#           _box_predictor=fake_box_predictor)
# ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
# ckpt.restore(checkpoint_path).expect_partial()

# # Run model through a dummy image so that variables are created
# image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
# prediction_dict = detection_model.predict(image, shapes)
# _ = detection_model.postprocess(prediction_dict, shapes)
# print('Weights restored!')

# # tf.keras.backend.clear_session()

# # print('Building model and restoring weights for fine-tuning.', flush=True)
# # #pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
# # #checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'
# # pipeline_config = 'models/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config'
# # checkpoint_path = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0'

# # # Load pipeling config and build a detection model
# # #
# # # Since we are working off of a COCO architecture which predicts 90
# # # class slots by default, we override the `num_classes` field here to be just one
# # configs = config_util.get_configs_from_pipeline_file(pipeline_config)
# # model_config = configs['model']
# # model_config.ssd.num_classes = num_classes
# # model_config.ssd.freeze_batchnorm = True
# # detection_model = model_builder.build(
# #     model_config=model_config, is_training=True
# # )

# # # Define checkpoints for desired layers
# # #
# # # We will now isolate the layers of `detection_model` that you wish to reuse so that you can
# # # restore the weights to just those layers.
# # # 1. Define checkpoints for the box predictor
# # # 2. Define checkpoints for the model, which will point to this box predictor checkpoint as well as the feature extraction layers
# # # 3. Restore the checkpoint for desired layers
# # fake_box_predictor = tf.compat.v2.train.Checkpoint(
# #     _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
# #     #_prediction_heads=detection_model._box_predictor._prediction_heads,
# #     _box_prediction_heads=detection_model._box_predictor._box_prediction_head,
# # )
# # fake_model = tf.compat.v2.train.Checkpoint(
# #     _feature_extractor=detection_model._feature_extractor,
# #     _box_predictor=fake_box_predictor
# # )
# # ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
# # ckpt.restore(checkpoint_path).expect_partial()

# # # Run model through a dummy image so that variables are created
# # image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
# # prediction_dict = detection_model.predict(image, shapes)
# # _ = detection_model.postprocess(prediction_dict, shapes)
# # print('Weights restored')

In [ ]:
# tf.keras.backend.clear_session()

# print('Building model and restoring weights for fine-tuning...', flush=True)
# num_classes = 1
# # pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
# # checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'
# # pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
# # checkpoint_path = '/content/DeepLogo2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'
# pipeline_config = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config'
# checkpoint_path = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0'


# # Load pipeline config and build a detection model.
# #
# # Since we are working off of a COCO architecture which predicts 90
# # class slots by default, we override the `num_classes` field here to be just
# # one (for our new rubber ducky class).
# configs = config_util.get_configs_from_pipeline_file(pipeline_config)
# model_config = configs['model']
# model_config.ssd.num_classes = num_classes
# model_config.ssd.freeze_batchnorm = True
# detection_model = model_builder.build(
#       model_config=model_config, is_training=True)

# # Set up object-based checkpoint restore --- RetinaNet has two prediction
# # `heads` --- one for classification, the other for box regression.  We will
# # restore the box regression head but initialize the classification head
# # from scratch (we show the omission below by commenting out the line that
# # we would add if we wanted to restore both heads)
# fake_box_predictor = tf.compat.v2.train.Checkpoint(
#     _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
#     # _prediction_heads=detection_model._box_predictor._prediction_heads,
#     #    (i.e., the classification head that we *will not* restore)
#     _box_prediction_head=detection_model._box_predictor._box_prediction_head,
#     )
# fake_model = tf.compat.v2.train.Checkpoint(
#           _feature_extractor=detection_model._feature_extractor,
#           _box_predictor=fake_box_predictor)
# ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
# ckpt.restore(checkpoint_path).expect_partial()

# # Run model through a dummy image so that variables are created
# image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
# prediction_dict = detection_model.predict(image, shapes)
# _ = detection_model.postprocess(prediction_dict, shapes)
# print('Weights restored!')

tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 1
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

## Training loop

In [ ]:
# tf.keras.backend.set_learning_phase(True)

In [ ]:
# # Inspect the layers of `detection_model`
# for i, v in enumerate(detection_model.trainable_variables):
#   pprint.pprint('i: {}, name: {}, shape: {}, dtype: {}'.format(i, v.name, v.shape, v.dtype))

In [ ]:
# # Select the prediction layers
# trainable_variables = detection_model.trainable_variables
# to_fine_tune = []
# prefixes_to_train = [
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
# for var in trainable_variables:
#   if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
#     to_fine_tune.append(var)

In [ ]:
# batch_size = 1
# learning_rate = 0.01
# num_batches = 100 * (len(train_images_np) // batch_size)

In [ ]:
# # Set up for fwd + bwd pass for a single train step
# def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
#   @tf.function
#   def train_step_fn(image_tensors,
#                     groundtruth_boxes_list,
#                     groundtruth_classes_list):
#     shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
#     model.provide_groundtruth(
#         groundtruth_boxes_list=groundtruth_boxes_list,
#         groundtruth_classes_list=groundtruth_classes_list
#     )
#     with tf.GradientTape() as tape:
#       preprocessed_images = tf.concat(
#           [detection_model.preprocess(image_tensor)[0]
#            for image_tensor in image_tensors], axis=0
#       )
#       prediction_dict = model.predict(preprocessed_images, shapes)
#       losses_dict = model.loss(prediction_dict, shapes)
#       total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
#       gradients = tape.gradient(total_loss, vars_to_fine_tune)
#       optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
#     return total_loss

#   return train_step_fn

In [ ]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
# train_step_fn = get_model_train_step_function(detection_model, optimizer, to_fine_tune)

# print('Start fine-tuning', flush=True)
# for idx in range(num_batches):
#   all_keys = list(range(len(train_images_np)))
#   random.shuffle(all_keys)
#   example_keys = all_keys[:batch_size]

#   gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
#   gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
#   image_tensors = [train_image_tensors[key] for key in example_keys]

#   total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

#   if idx != 0 and idx % 10 == 0:
#     print('batch ' + str(idx) + ' of ' + str(num_batches) + ', loss=' + str(total_loss.numpy()), flush=True)

# print('Done fine-tuning')

In [ ]:
# tf.keras.backend.set_learning_phase(True)

# # These parameters can be tuned; since our training set has 5 images
# # it doesn't make sense to have a much larger batch size, though we could
# # fit more examples in memory if we wanted to.
# batch_size = 4
# learning_rate = 0.01
# num_batches = 100

# # Select variables in top layers to fine-tune.
# trainable_variables = detection_model.trainable_variables
# to_fine_tune = []
# prefixes_to_train = [
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
# for var in trainable_variables:
#   if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
#     to_fine_tune.append(var)

# # Set up forward + backward pass for a single train step.
# def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
#   """Get a tf.function for training step."""

#   # Use tf.function for a bit of speed.
#   # Comment out the tf.function decorator if you want the inside of the
#   # function to run eagerly.
#   @tf.function
#   def train_step_fn(image_tensors,
#                     groundtruth_boxes_list,
#                     groundtruth_classes_list):
#     """A single training iteration.

#     Args:
#       image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
#         Note that the height and width can vary across images, as they are
#         reshaped within this function to be 640x640.
#       groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
#         tf.float32 representing groundtruth boxes for each image in the batch.
#       groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
#         with type tf.float32 representing groundtruth boxes for each image in
#         the batch.

#     Returns:
#       A scalar tensor representing the total loss for the input batch.
#     """
#     shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
#     model.provide_groundtruth(
#         groundtruth_boxes_list=groundtruth_boxes_list,
#         groundtruth_classes_list=groundtruth_classes_list)
#     with tf.GradientTape() as tape:
#       preprocessed_images = tf.concat(
#           [detection_model.preprocess(image_tensor)[0]
#            for image_tensor in image_tensors], axis=0)
#       prediction_dict = model.predict(preprocessed_images, shapes)
#       losses_dict = model.loss(prediction_dict, shapes)
#       total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
#       gradients = tape.gradient(total_loss, vars_to_fine_tune)
#       optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
#     return total_loss

#   return train_step_fn

# optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
# train_step_fn = get_model_train_step_function(
#     detection_model, optimizer, to_fine_tune)

# print('Start fine-tuning!', flush=True)
# for idx in range(num_batches):
#   # Grab keys for a random subset of examples
#   all_keys = list(range(len(train_images_np)))
#   random.shuffle(all_keys)
#   example_keys = all_keys[:batch_size]
#   print(example_keys)

#   # Note that we do not do data augmentation in this demo.  If you want a
#   # a fun exercise, we recommend experimenting with random horizontal flipping
#   # and random cropping :)
#   gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
#   gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
#   image_tensors = [train_image_tensors[key] for key in example_keys]

#   # Training step (forward pass + backwards pass)
#   total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

#   if idx % 10 == 0:
#     print('batch ' + str(idx) + ' of ' + str(num_batches)
#     + ', loss=' +  str(total_loss.numpy()), flush=True)

# print('Done fine-tuning!')

In [ ]:
# tf.keras.backend.set_learning_phase(True)

# # These parameters can be tuned; since our training set has 5 images
# # it doesn't make sense to have a much larger batch size, though we could
# # fit more examples in memory if we wanted to.
# batch_size = 4
# learning_rate = 0.01
# num_batches = 100

# # Select variables in top layers to fine-tune.
# trainable_variables = detection_model.trainable_variables
# to_fine_tune = []
# prefixes_to_train = [
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
#   'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
# for var in trainable_variables:
#   if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
#     to_fine_tune.append(var)

# # Set up forward + backward pass for a single train step.
# def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
#   """Get a tf.function for training step."""

#   # Use tf.function for a bit of speed.
#   # Comment out the tf.function decorator if you want the inside of the
#   # function to run eagerly.
#   @tf.function
#   def train_step_fn(image_tensors,
#                     groundtruth_boxes_list,
#                     groundtruth_classes_list):
#     """A single training iteration.

#     Args:
#       image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
#         Note that the height and width can vary across images, as they are
#         reshaped within this function to be 640x640.
#       groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
#         tf.float32 representing groundtruth boxes for each image in the batch.
#       groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
#         with type tf.float32 representing groundtruth boxes for each image in
#         the batch.

#     Returns:
#       A scalar tensor representing the total loss for the input batch.
#     """
#     shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
#     model.provide_groundtruth(
#         groundtruth_boxes_list=groundtruth_boxes_list,
#         groundtruth_classes_list=groundtruth_classes_list)
#     with tf.GradientTape() as tape:
#       preprocessed_images = tf.concat(
#           [detection_model.preprocess(image_tensor)[0]
#            for image_tensor in image_tensors], axis=0)
#       prediction_dict = model.predict(preprocessed_images, shapes)
#       losses_dict = model.loss(prediction_dict, shapes)
#       total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
#       gradients = tape.gradient(total_loss, vars_to_fine_tune)
#       optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
#     return total_loss

#   return train_step_fn

# optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
# train_step_fn = get_model_train_step_function(
#     detection_model, optimizer, to_fine_tune)

# print('Start fine-tuning!', flush=True)
# for idx in tqdm(range(num_batches)):
#   # Grab keys for a random subset of examples
#   all_keys = list(range(len(train_images_np)))
#   random.shuffle(all_keys)
#   example_keys = all_keys[:batch_size]

#   # Note that we do not do data augmentation in this demo.  If you want a
#   # a fun exercise, we recommend experimenting with random horizontal flipping
#   # and random cropping :)
#   gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
#   gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
#   image_tensors = [train_image_tensors[key] for key in example_keys]

#   # Training step (forward pass + backwards pass)
#   total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

#   if idx != 0 and idx % 10 == 0:
#     print('batch ' + str(idx) + ' of ' + str(num_batches)
#     + ', loss=' +  str(total_loss.numpy()), flush=True)

# print('Done fine-tuning!')

tf.keras.backend.set_learning_phase(True)

# These parameters can be tuned; since our training set has 5 images
# it doesn't make sense to have a much larger batch size, though we could
# fit more examples in memory if we wanted to.
batch_size = 4
learning_rate = 0.01
num_batches = 100

# Select variables in top layers to fine-tune.
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
  if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
    to_fine_tune.append(var)

# Set up forward + backward pass for a single train step.
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
  """Get a tf.function for training step."""

  # Use tf.function for a bit of speed.
  # Comment out the tf.function decorator if you want the inside of the
  # function to run eagerly.
  @tf.function
  def train_step_fn(image_tensors,
                    groundtruth_boxes_list,
                    groundtruth_classes_list):
    """A single training iteration.

    Args:
      image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    with tf.GradientTape() as tape:
      preprocessed_images = tf.concat(
          [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_tensors], axis=0)
      prediction_dict = model.predict(preprocessed_images, shapes)
      losses_dict = model.loss(prediction_dict, shapes)
      total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
      gradients = tape.gradient(total_loss, vars_to_fine_tune)
      optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    return total_loss

  return train_step_fn

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(
    detection_model, optimizer, to_fine_tune)

print('Start fine-tuning!', flush=True)
for idx in range(num_batches):
  # Grab keys for a random subset of examples
  all_keys = list(range(len(train_images_np)))
  random.shuffle(all_keys)
  example_keys = all_keys[:batch_size]

  # Note that we do not do data augmentation in this demo.  If you want a
  # a fun exercise, we recommend experimenting with random horizontal flipping
  # and random cropping :)
  gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
  gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
  image_tensors = [train_image_tensors[key] for key in example_keys]

  # Training step (forward pass + backwards pass)
  total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

  if idx % 10 == 0:
    print('batch ' + str(idx) + ' of ' + str(num_batches)
    + ', loss=' +  str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')

## Run Test

In [ ]:
# test_annot_csv = 'flickr_logos_27_dataset/flickr_logos_27_dataset_test_set_annotation_cropped.txt'
# test_img_dir = 'flickr_logos_27_dataset/flickr_logos_27_dataset_images'
# test_images_np = []
# csvs = np.loadtxt(test_annot_csv, dtype=str, delimiter=',')
# for csv in tqdm(csvs):
#   if csv[-1] != '0':
#     continue
#   img_fname = csv[0]
#   image_path = os.path.join(test_img_dir, img_fname)
#   test_images_np.append(np.expand_dims(
#       load_image_into_numpy_array(image_path), axis=0)
#   )

In [ ]:
# for idx, test_image_np in enumerate(test_images_np[:5]):
#   plt.subplot(2, 3, idx + 1)
#   plt.imshow(test_image_np[0])
# plt.show()

In [ ]:
# gc.collect()

In [ ]:
# @tf.function
# def detect(input_tensor):
#   preprocessed_image, shapes = detection_model.preprocess(input_tensor)
#   prediction_dict = detection_model.predict(preprocessed_image, shapes)
#   return detection_model.postprocess(prediction_dict, shapes)

In [ ]:
# # Note that the first frame will trigger tracing of the tf.function, which will
# # take some time, after which inference should be fast.
# label_id_offset = 1
# np.random.shuffle(test_images_np)

In [ ]:
# test_idx = np.random.choice(len(test_images_np))
# print(test_idx)

In [ ]:
# input_tensor = tf.convert_to_tensor(test_images_np[test_idx], dtype=np.float32)
# detections = detect(input_tensor)

In [ ]:
# print(max(detections['detection_scores'][0]))
# print(detections['detection_scores'])

In [ ]:
# image_np_with_annotations = test_images_np[test_idx].copy()

In [ ]:
# image_np_with_annotations.shape

In [ ]:
# viz_utils.visualize_boxes_and_labels_on_image_array(
#     image_np_with_annotations[0],
#     detections['detection_boxes'][0].numpy(), 
#     detections['detection_classes'][0].numpy().astype(np.uint32) + label_id_offset, 
#     detections['detection_scores'][0].numpy(), category_index,
#     use_normalized_coordinates=True,
#     min_score_thresh=0.7
# )
# plt.imshow(image_np_with_annotations[0])
# plt.show()

In [ ]:
test_image_dir = 'models/research/object_detection/test_images/ducky/test/'
test_images_np = []
for i in range(1, 50):
  image_path = os.path.join(test_image_dir, 'out' + str(i) + '.jpg')
  test_images_np.append(np.expand_dims(
      load_image_into_numpy_array(image_path), axis=0))

# Again, uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):
  """Run detection on an input image.

  Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

  Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
  """
  preprocessed_image, shapes = detection_model.preprocess(input_tensor)
  prediction_dict = detection_model.predict(preprocessed_image, shapes)
  return detection_model.postprocess(prediction_dict, shapes)

# Note that the first frame will trigger tracing of the tf.function, which will
# take some time, after which inference should be fast.

label_id_offset = 1
for i in range(len(test_images_np)):
  input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
  detections = detect(input_tensor)

  plot_detections(
      test_images_np[i][0],
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.uint32)
      + label_id_offset,
      detections['detection_scores'][0].numpy(),
      category_index, figsize=(15, 20), image_name="gif_frame_" + ('%02d' % i) + ".jpg")

In [ ]:
imageio.plugins.freeimage.download()

anim_file = 'duckies_test.gif'

filenames = glob.glob('gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []
for filename in filenames:
  image = imageio.imread(filename)
  images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=5)

display(IPyImage(open(anim_file, 'rb').read()))